In [1]:
%reload_ext autoreload
%autoreload 2

import os, pickle, sys
import numpy as np
import pandas as pd
import main_funcs as mfun
import utils_funcs as utils
import plot_funcs as pfun
import matplotlib.pyplot as plt
from scipy import stats
import glob
from scipy.signal import resample
from scipy.cluster.hierarchy import linkage, leaves_list
import warnings
warnings.filterwarnings("ignore", category=RuntimeWarning) 
warnings.filterwarnings("ignore", category= FutureWarning) 
warnings.filterwarnings("ignore", category= DeprecationWarning)
warnings.filterwarnings("ignore", category= UserWarning)

pfun.set_figure()

infoPath = 'C:\\Users\\Huriye\\Documents\\code\\clapfcstimulation\\analysis\\infoForAnalysis.pkl'
info = pd.read_pickle(infoPath) 

## Parameters
fRate = 1000/30
pre_frames, post_frames, analysisWindowDur, simulationDur = pfun.set_analysisParams ()
responsiveness_test_duration = 1000.0 #in ms
simulationDur_ms = 350.0 # in ms 
simulationDur  = int(np.ceil(simulationDur_ms/fRate))
shutterLength  = int(np.round(simulationDur_ms/fRate))
tTypes = ['All','Both', 'onlyOpto', 'onlyVis']

Env: clapfcstimulation


In [ ]:
# Loop through all recordings - MUST BE RUN TO UPDATE THE INFO FILE
# For each recording one file saved, each index is a cell 

for ind, recordingDate in enumerate(info.recordingList.recordingDate):
   savepathname = info.recordingList.analysispathname[ind]
   filenamePAQ_analysis = [f for f in glob.glob(savepathname + 'extracted_variables.pkl')] # paq analysis file
   if  (info.recordingList.calciumImaging[ind] ==0) & (info.recordingList.pupilImaging[ind] ==0):
      info.recordingList.loc[ind,'extractedVar'] = 0
   elif (len(filenamePAQ_analysis)==1):
      info.recordingList.loc[ind,'extractedVar'] = 1 
   elif (info.recordingList.stimuliFamiliarity[ind] == 5):
      info.recordingList.loc[ind,'extractedVar'] = 0
   elif (info.recordingList.stimuliFamiliarity[ind] == 10):
      info.recordingList.loc[ind,'extractedVar'] = 0
   else:
      try:
      #if ind ==125:
         print(str(ind) + ': Creating: ' + info.recordingList.analysispathname[ind])
         #Create a huge dictionary with all cells and parameters for each cell
         pathname = info.recordingList.analysispathname[ind]
         print('Creating the dict for:' + pathname)

         ########## Organise stimuli times 
         if  (info.recordingList.paqExtraction[ind] ==1) :
            paqData = pd.read_pickle (pathname +'paq-data.pkl')
            paqRate = paqData['rate']
            # Get the stim start times 
            frame_clock    = utils.paq_data (paqData, 'prairieFrame', threshold_ttl=True, plot=False)
            optoStimTimes  = utils.paq_data (paqData, 'optoLoopback', threshold_ttl=True, plot=False)
            if   (len(optoStimTimes)>500):

               print('Opto stim times is a lot , check it out : ' + str(ind))
            else: # the frame_clock is slightly longer in paq as there are some up to a sec delay from
               # microscope to PAQI/O software.  
               optoStimTimes = utils.stim_start_frame (paq=paqData, stim_chan_name='optoLoopback',
                                                   frame_clock=None,stim_times=None, plane=0, n_planes=1)
               visStimTimes = utils.stim_start_frame (paq=paqData, stim_chan_name='maskerLED',
                                                   frame_clock=None,stim_times=None, plane=0, n_planes=1)
               shutterTimes = utils.shutter_start_frame (paq=paqData, stim_chan_name='shutterLoopback',
                                                   frame_clock=None,stim_times=None, plane=0, n_planes=1)
               # Lets organise it more for analysis friendly format
               trialStartTimes = np.unique(np.concatenate((optoStimTimes,visStimTimes),0))
               # final check if there is a problematic stim start
               first_ind = np.where(np.diff(trialStartTimes)>30) # should be at least one sec between stim starts
               first_ind = np.concatenate(([0], first_ind[0]+1))
               trialStartTimes = np.array(trialStartTimes)
               trialStartTimes = trialStartTimes[first_ind]
               trialTypes = []
               reree
               for t in trialStartTimes:
                  optoexist =  np.any(optoStimTimes== t)
                  visexist  =  np.any( visStimTimes == t)
                  if  optoexist  & visexist: 
                     trialTypes += ['Both']
                  elif optoexist &~ visexist:
                     trialTypes += ['onlyOpto']
                  elif ~optoexist & visexist:
                     trialTypes += ['onlyVis']
                  else:
                     trialTypes += ['CHECK']
               trialStartTimes = shutterTimes
               #t = [idx for idx, t_type in enumerate(trialTypes) if t_type=='Both']
            
            ########## Organise calcium imaging traces 
            pvals ={} 
            pvalsWilcoxon = {}
            dffTrace ={} 
            dffTrace_mean ={}
            dffTrace_median ={}
            dffAfterStim1500ms_mean ={}
            dffTrace_normalised = {}
            dffTrace_mean_normalised ={}
            
            if (info.recordingList.dffExtraction[ind] ==1):
               imData = pd.read_pickle (pathname +'imaging-data.pkl')
               fluR      = imData['flu']
               n_frames  = imData['n_frames']
               flu_raw   = imData['flu_raw']

               # Lets put nans for stimulated frames
               frameTimes = np.full((1,fluR.shape[1] ), False) # create a full false array
               for sT in shutterTimes:
                  frameTimes[:,sT:(sT+shutterLength)] = True
               fluR[:, frameTimes[0,:]] = np.nan

               # clean detrended traces
               flu = utils.clean_traces(fluR)
               flu_normalised = mfun.norm_to_zero_one (flu)

               ### Look for responsiveness for 4 trial types
               for t in tTypes:
                  if t =='All':
                     trialInd = np.transpose(list(range(len(trialStartTimes))))
                  else:
                     trialInd = [idx for idx, t_type in enumerate(trialTypes) if t_type==t]
                  
                  if len(trialInd)>1:
                     _, _, pval = utils.test_responsive  (flu, frame_clock, trialStartTimes[trialInd], pre_frames=int(np.ceil(responsiveness_test_duration/fRate)), 
                                                               post_frames=int(np.ceil(responsiveness_test_duration/fRate)), offset=simulationDur,
                                                               testType ='ttest')
                     pvals[t] = pval
                     _, _, pval = utils.test_responsive  (flu, frame_clock, trialStartTimes[trialInd], pre_frames=int(np.ceil(responsiveness_test_duration/fRate)), 
                                                               post_frames=int(np.ceil(responsiveness_test_duration/fRate)), offset=simulationDur,
                                                               testType ='wilcoxon')
                     pvalsWilcoxon[t] = pval
               nCell = len(flu)
               print('number of cell: ' + str(nCell))
               ### Get dff values for 4 trial types
               for indx, t in enumerate(tTypes) :
                  if t =='All':
                     trialInd = np.transpose(list(range(len(trialStartTimes))))
                  else:
                     trialInd = [idx for idx, t_type in enumerate(trialTypes) if t_type==t]
                  
                  if len(trialInd)>1:
                     dffTrace[t]      = utils.flu_splitter(flu, trialStartTimes[trialInd], pre_frames, post_frames) # Cell x time x trial
                     dffTrace_mean[t] = np.mean(dffTrace[t],2) # Cell x time
                     dffTrace_median[t] = np.median(dffTrace[t],2) # Cell x time
                     dffAfterStim1500ms_mean[t] = np.nanmean(dffTrace_mean[t][:, (pre_frames+simulationDur): (pre_frames + analysisWindowDur)],1)
                     
                     dffTrace_normalised[t] = utils.flu_splitter(flu_normalised, trialStartTimes[trialInd], pre_frames, post_frames) # Cell x time x trial
                     dffTrace_mean_normalised[t] = np.mean (dffTrace_normalised[t],2) # Cell x time ( mean - 2)
                           #create dff for all cells
               # PLOT EACH CELL RESPONSE
               for indC, cell_dff in enumerate(flu):
                  plotTrue = True
                  if (len(optoStimTimes)==40): 
                     tTypes = ['Both', 'onlyOpto', 'onlyVis']
                     plotTrue =  (pvals['onlyVis'][indC] < 1e-6) or (pvals['onlyOpto'][indC] < 1e-6) or (pvals['Both'][indC] < 1e-6)
                  elif (len(optoStimTimes)==20):
                     tTypes = ['Both', 'onlyVis']
                     plotTrue =  (pvals['onlyVis'][indC] < 1e-6)  or (pvals['Both'][indC] < 1e-6)

                  if plotTrue:
                     fig = plt.figure(str(indC))
                     plt.switch_backend('Agg')
                     for indx, t in enumerate(tTypes):
                           pfun.lineplot_withSEM (data=dffTrace[t][indC], colorInd = indx, label=t)
                           pfun.save_figure(pathname[-20:-1]+'_DFF_mean_' + str(indC),info.figsPath +'\\cellDFF_Chrimson')
                           plt.close(fig)
            else:
               imData =[]
            ########## Organise pupil  traces 
            pupilTrace ={}
            pupilTrace_mean ={}
            pupilTraceVer ={}
            pupilTrace_meanVer ={}

            if (info.recordingList.pupilExtraction[ind] ==1): 
               pupilData = pd.read_pickle (pathname +'pupil-data.pkl')
               pupilrawh = pupilData['horizontalDis']
               pupilrawv = pupilData['verticalDis']
               pupilQualityh = pupilData['horizontallikelihood']
               pupilQualityv = pupilData['verticallikelihood']

               for indx, t in enumerate(tTypes) :
                  if t =='All':
                     trialInd = np.transpose(list(range(len(trialStartTimes))))
                  else:
                     trialInd = [idx for idx, t_type in enumerate(trialTypes) if t_type==t]
                  
                  if len(trialInd)>1:
                     pupilTrace[t]  = utils.trace_splitter(pupilrawh, trialStartTimes[trialInd], pre_frames, post_frames) # Cell x time x trial
                     pupilTrace_mean[t]  = np.mean(pupilTrace[t],2) # Cell x time

                     pupilTraceVer[t]  = utils.trace_splitter(pupilrawv, trialStartTimes[trialInd], pre_frames, post_frames) # Cell x time x trial
                     pupilTrace_meanVer[t]  = np.mean(pupilTraceVer[t],2) # Cell x tim

            #lets get recording info
            animalID = []
            x_coordinate = []
            y_coordinate = []
            z_coordinate = []
            stimuliFamilarity = []
            dataQuality =[]
            recData = []
            recID   = []
            cellID  = []

            if len(imData)>0:
               imStats = imData['stat']
               for idx, cell_flu in enumerate(flu): # from suite2p website: med: (y,x) center of cell
                  x_coordinate += [np.round(imStats[idx]['med'][1] *512/558.9) + info.recordingList['x-coordinate'] [ind]]
                  y_coordinate += [np.round(imStats[idx]['med'][0] *512/558.9) + info.recordingList['y-coordinate'] [ind]]
                  z_coordinate += [info.recordingList['z-coordinate'] [ind]]
                  animalID     += [info.recordingList['animalID'] [ind]]
                  stimuliFamilarity += [info.recordingList['stimuliFamiliarity'] [ind]]
                  dataQuality  += [info.recordingList['dataQuality'] [ind]]
                  recData  += [info.recordingList['recordingDate'] [ind]]
                  recID    += [info.recordingList['recordingID'] [ind]]
                  cellID   += [idx]
            else:
               x_coordinate = [np.nan]
               y_coordinate = [np.nan]
               z_coordinate = [np.nan]
               animalID     = [info.recordingList['animalID'] [ind]]
               stimuliFamilarity = [info.recordingList['stimuliFamiliarity'] [ind]]
               dataQuality  = [info.recordingList['dataQuality'] [ind]]
               recData  = [info.recordingList['recordingDate'] [ind]]
               recID    = [info.recordingList['recordingID'] [ind]]
               cellID   = [np.nan]

            #save outputs for population analysis
            savename = info.recordingList.analysispathname[ind] + 'extracted_variables.pkl'
            with open(savename, 'wb') as f:  # Python 3: open(..., 'wb')
               pickle.dump([trialStartTimes, trialTypes, pvals, dffTrace, 
                           dffTrace_mean, dffAfterStim1500ms_mean,
                           x_coordinate,y_coordinate,z_coordinate,
                           animalID,stimuliFamilarity,pvalsWilcoxon, dataQuality,
                           recData, recID, cellID, pupilTrace, pupilTrace_mean, 
                           pupilQualityh, pupilTraceVer, pupilTrace_meanVer, pupilQualityv,
                           dffTrace_normalised,dffTrace_mean_normalised, dffTrace_median], f)
            info.recordingList.loc[ind,'extractedVar'] = 1
         else:
            print('FAILED: no paq file : ' + pathname)
            info.recordingList.loc[ind,'extractedVar'] = 0
      except:
         print('FAILED: ' + pathname)
         info.recordingList.loc[ind,'extractedVar'] = 0
   

#save info 
filenameINFO = info.analysisPath + '\\infoForAnalysis-extracted.pkl'
with open(filenameINFO, 'wb') as f:
    pickle.dump(info, f)
print('All should be done!!')

In [ ]:
# Version: 31/03/34 - Optimised merge code
## Merge all datapoints & save them in a dictionary

infoPath = 'C:\\Users\\Huriye\\Documents\\code\\clapfcstimulation\\analysis\\infoForAnalysis-extracted.pkl'
info = pd.read_pickle(infoPath)

# Initialize empty or None types for variables to be aggregated.
dff_traceBoth, dff_traceVis, dff_traceOpto = None, None, None
dff_traceBoth_median, dff_traceVis_median, dff_traceOpto_median = None, None, None
dff_traceBoth_normalised, dff_traceVis_normalised, dff_traceOpto_normalised = None, None, None
pupil_traceBoth, pupil_traceVis, pupil_traceOpto = None, None, None
pupilID, x_coordinate, y_coordinate, z_coordinate = [], [], [], []
animalID, stimuliFamilarity, dataQuality, recData, recID, cellID = [], [], [], [], [], []
pvalsBoth, pvalsVis, pvalsOpto = [], [], []
dff_meanBothValue, dff_meanVisValue, dff_meanOptoValue = [], [], []

# Create the main variables for plots by merging the extracted variables from all recordings
ty = 0
indPupil = 0
for ind, recordingDate in enumerate(info.recordingList.recordingDate):
   if (info.recordingList.extractedVar[ind] ==1):
        # load the extracted values
        pathname = info.recordingList.analysispathname[ind] 
        extData = pd.read_pickle (pathname + 'extracted_variables.pkl')
        sys.stdout.write(f'\rExtraction started for : {ind}')
        sys.stdout.flush() 

        len_cellID   = len(extData[15])
        if len_cellID==0:
            len_cellID=1

        # Get Dff traces MEAN
        dff_trace = extData[4] #4 for mean, 24 for median
        shape = (len_cellID, 240) 
        dff_traceBoth =mfun.update_dff_traces(dff_traceBoth, dff_trace, 'Both', shape)
        dff_traceVis = mfun.update_dff_traces(dff_traceVis, dff_trace, 'onlyVis', shape)
        dff_traceOpto = mfun.update_dff_traces(dff_traceOpto, dff_trace, 'onlyOpto', shape)

        # Get Dff traces MEDIAN
        dff_trace = extData[24] #4 for mean, 24 for median
        shape = (len_cellID, 240) 
        dff_traceBoth_median =mfun.update_dff_traces(dff_traceBoth_median, dff_trace, 'Both', shape)
        dff_traceVis_median = mfun.update_dff_traces(dff_traceVis_median, dff_trace, 'onlyVis', shape)
        dff_traceOpto_median = mfun.update_dff_traces(dff_traceOpto_median, dff_trace, 'onlyOpto', shape)

        # Get Dff traces NORMALISED
        dff_trace = extData[23] #4 for mean, 24 for median
        shape = (len_cellID, 240)
        dff_traceBoth_normalised =mfun.update_dff_traces(dff_traceBoth_normalised, dff_trace, 'Both', shape)
        dff_traceVis_normalised = mfun.update_dff_traces(dff_traceVis_normalised, dff_trace, 'onlyVis', shape)
        dff_traceOpto_normalised = mfun.update_dff_traces(dff_traceOpto_normalised, dff_trace, 'onlyOpto', shape)

        # Get Pupil traces
        indPupil, pupilID, pupil_traceBoth, pupil_traceVis, pupil_traceOpto = mfun.update_pupil_traces(
            extData, len_cellID, indPupil, pupilID, pupil_traceBoth, pupil_traceVis, pupil_traceOpto)
        
        # Get P values

        pvals = extData[2]
        if len(pvals)>0:
            pvalsBoth += pvals['Both'].tolist()
            pvalsVis  += pvals['onlyVis'].tolist()

            if 'onlyOpto' in dff_trace:
                pvalsOpto  += pvals['onlyOpto'].tolist()
            else:
                fakeOpto = np.empty (np.shape(pvals['Both']))
                fakeOpto[:] = 5
                pvalsOpto  += fakeOpto.tolist()
        else:
            pvalsBoth += [5]
            pvalsVis  += [5]
            pvalsOpto += [5]

         
        # Get  cell-related information
        x_coordinate += extData[6]
        y_coordinate += extData[7]
        z_coordinate += extData[8]
        animalID     += extData[9]
        stimuliFamilarity += extData[10]
        dataQuality += extData[12]
        recData  += extData[13]
        recID    += extData[14]
        cellID   += extData[15]

########################################
####################### Organise & save files

fRate = 1000/30.0
pre_frames    = 2000.0# in ms
pre_frames    = int(np.ceil(pre_frames/fRate))
post_frames   = 6000.0 # in ms
post_frames   = int(np.ceil(post_frames/fRate))
analysis_time = 1500.0 # in ms
analysis_time = int(np.ceil(analysis_time/fRate))
simulationDur_ms = 350.0 # in ms
simulationDur = int(np.ceil(simulationDur_ms/fRate))
pupil_resample_num = int(6*5)

### Lets normalise to baseline - MEAN
traces = {  'Vis': dff_traceVis,
            'Opto': dff_traceOpto,
            'Both': dff_traceBoth}

dff_traceVis_normalisedtopre, dff_traceOpto_normalisedtopre, dff_traceBoth_normalisedtopre = (
    mfun.normalize_to_baseline(traces[key], pre_frames) for key in traces)
### Lets do tiny cleaning for imaging traces -MEDIAN
traces = {  'Vis': dff_traceVis_median,
            'Opto': dff_traceOpto_median,
            'Both': dff_traceBoth_median}
dff_traceVis_normalisedtopre_median, dff_traceOpto_normalisedtopre_median, dff_traceBoth_normalisedtopre_median = (
    mfun.normalize_to_baseline(traces[key], pre_frames) for key in traces)

### Lets do tiny cleaning for pupil traces
traces = {  'Vis': pupil_traceVis,
            'Opto': pupil_traceOpto,
            'Both': pupil_traceBoth}
pupil_traceVis, pupil_traceOpto, pupil_traceBoth = (
    mfun.normalize_to_baseline(traces[key], pre_frames) for key in traces)

pupil_traceOpto = resample(pupil_traceOpto, pupil_resample_num, axis=1)
pupil_traceVis = resample(pupil_traceVis, pupil_resample_num, axis=1)
pupil_traceBoth = resample(pupil_traceBoth, pupil_resample_num, axis=1)

#  More version of dff traces
dff_meanBoth = np.nanmean(dff_traceBoth [:, pre_frames:(pre_frames + simulationDur + analysis_time)],axis=1)
dff_meanVis  = np.nanmean(dff_traceVis  [:, pre_frames:(pre_frames + simulationDur + analysis_time)],axis=1)
dff_meanOpto = np.nanmean(dff_traceOpto [:, pre_frames:(pre_frames + simulationDur + analysis_time)],axis=1) 

zdff_traceBoth = stats.zscore (dff_traceBoth, nan_policy='omit')
zdff_traceVis  = stats.zscore (dff_traceVis, nan_policy='omit')
zdff_traceOpto = stats.zscore (dff_traceOpto, nan_policy='omit')

filenameINFO = info.analysisPath + '\\infoForAnalysis-readyForSelectingInterestedCells.pkl'
with open(filenameINFO, 'wb') as f:
    pickle.dump((animalID, stimuliFamilarity, dataQuality,
                 recData, recID, cellID, 
                 pvalsBoth, pvalsVis, pvalsOpto,
                 dff_meanVis, dff_meanBoth, 
                 dff_meanOpto, pupilID),f)

filenameINFO = info.analysisPath + '\\infoForAnalysis-readyForPlotting.pkl'
with open(filenameINFO, 'wb') as f:
    pickle.dump((dff_traceBoth, dff_traceVis, dff_traceOpto), f)
    
filenameINFO = info.analysisPath + '\\infoForAnalysis-readyForPlotting_median.pkl'
with open(filenameINFO, 'wb') as f:
    pickle.dump((dff_traceBoth_median, dff_traceVis_median, dff_traceOpto_median), f)
    
filenameINFO = info.analysisPath + '\\infoForAnalysis-readyForPlotting_normalised.pkl'
with open(filenameINFO, 'wb') as f:
    pickle.dump((dff_traceBoth_normalised, dff_traceVis_normalised, dff_traceOpto_normalised), f)
    
filenameINFO = info.analysisPath + '\\infoForAnalysis-readyForPlotting_normalisedtoPre.pkl'
with open(filenameINFO, 'wb') as f:
    pickle.dump((dff_traceBoth_normalisedtopre, dff_traceVis_normalisedtopre, dff_traceOpto_normalisedtopre), f)
    
filenameINFO = info.analysisPath + '\\infoForAnalysis-readyForPlotting_normalisedtoPre_median.pkl'
with open(filenameINFO, 'wb') as f:
    pickle.dump((dff_traceBoth_normalisedtopre_median, dff_traceVis_normalisedtopre_median, dff_traceOpto_normalisedtopre_median), f)
    
filenameINFO = info.analysisPath + '\\infoForAnalysis-readyForPlotting_position.pkl'
with open(filenameINFO, 'wb') as f:
    pickle.dump((x_coordinate, y_coordinate, z_coordinate ), f)

filenameINFO = info.analysisPath + '\\infoForAnalysis-readyForPlottingPupil.pkl'
with open(filenameINFO, 'wb') as f:
    pickle.dump((pupil_traceVis, pupil_traceBoth, pupil_traceOpto ), f)
print('\nAll should be done!!')
    


In [ ]:
# More stats

infoPath = 'C:\\Users\\Huriye\\Documents\\code\\clapfcstimulation\\analysis\\infoForAnalysis-extracted.pkl'
info = pd.read_pickle(infoPath)

# Create the main variables for plots by merging the extracted variables from all recordings
pre_frames, post_frames, analysis_frame, simulationDur = pfun.set_analysisParams ()
pre_analysisWindow = np.arange(pre_frames - analysis_frame, pre_frames)
post_analysisWindow = np.arange((pre_frames+simulationDur), (pre_frames + simulationDur + analysis_frame))

# Create dictionaries to store the values
variance_dict_pre = {'Both': [], 'onlyVis': [], 'onlyOpto': []}
variance_dict_post = {'Both': [], 'onlyVis': [], 'onlyOpto': []}
mi_dict = {'Both': [], 'onlyVis': [], 'onlyOpto': []}
snr_dict = {'Both': [], 'onlyVis': [], 'onlyOpto': []}
abs_dict = {'Both': [], 'onlyVis': [], 'onlyOpto': []}
crosscorr_dict_pre = {'Both': [], 'onlyVis': [], 'onlyOpto': []}
crosscorr_dict_post = {'Both': [], 'onlyVis': [], 'onlyOpto': []}

for ind, recordingDate in enumerate(info.recordingList.recordingDate):
    if info.recordingList.extractedVar[ind] == 1:
        # Load the extracted values
        pathname = info.recordingList.analysispathname[ind]
        extData = pd.read_pickle(pathname + 'extracted_variables.pkl')
        sys.stdout.write(f'\rMore stats are calculating for : {ind}')
        sys.stdout.flush() 
        len_cellID = len(extData[15])

        # Loop through the datasets ('Both', 'onlyVis', 'onlyOpto')
        for dataset in ['Both', 'onlyVis', 'onlyOpto']:
            dff_trace = extData[3]
            if dataset in dff_trace:
                flu = dff_trace[dataset]
                # Calculate variance
                variance_value_pre = mfun.variance_cell_rates(flu, pre_analysisWindow)
                variance_dict_pre[dataset] += variance_value_pre.tolist()
                variance_value_post = mfun.variance_cell_rates(flu, post_analysisWindow)
                variance_dict_post[dataset] += variance_value_post.tolist()

                # Calculate cross-correlation
                crosscorr_value_post = mfun.mean_cross_correlation(flu, pre_analysisWindow)
                crosscorr_dict_pre[dataset] += crosscorr_value_post.tolist()

                crosscorr_value_post = mfun.mean_cross_correlation(flu, post_analysisWindow)
                crosscorr_dict_post[dataset] += crosscorr_value_post.tolist()

                # Calculate SNR
                snr_value = mfun.calculate_SNR(flu, pre_analysisWindow, post_analysisWindow)
                snr_dict[dataset] += snr_value.tolist()

                # Calculate Absolute Value
                abs_value = mfun.calculate_absMagnitude(flu, pre_analysisWindow, post_analysisWindow)
                abs_dict[dataset] += abs_value.tolist()

                # Calculate MI (if applicable)
                mi_value = mfun.calculate_MI(flu, pre_analysisWindow, post_analysisWindow)
                mi_dict[dataset] += mi_value.tolist()
            else:
                if 'onlyOpto' in dff_trace:
                    fakeOpto = np.empty(np.shape(dff_trace['Both']))
                else:
                    fakeOpto = np.empty(1)
                fakeOpto[:] = np.nan

                variance_dict_pre[dataset] += fakeOpto.tolist()
                variance_dict_post[dataset] += fakeOpto.tolist()
                crosscorr_dict_pre[dataset] += fakeOpto.tolist()
                crosscorr_dict_post[dataset] += fakeOpto.tolist()
                snr_dict[dataset] += fakeOpto.tolist()
                abs_dict[dataset] += fakeOpto.tolist()
                mi_dict[dataset] += fakeOpto.tolist()

filenameINFO = info.analysisPath + '\\infoForAnalysis-readyForPlotting_moreStats.pkl'
with open(filenameINFO, 'wb') as f:
    pickle.dump((variance_dict_pre, variance_dict_post, snr_dict, mi_dict,
                 crosscorr_dict_pre,crosscorr_dict_post, abs_dict), f)
    
print('\nCompleted')

In [3]:
#Cross correlation matrix extraction

interestedCohorts = [ 'Chrimson']
trainedLevels = [ 'Trained']
responsivenessTypes = ['All', 'None']
stimTypes = ['Visual', 'Visual + Opto']
sortType = 'Visual'
fRate = 30
simulationDur_ms = 350.0 # in ms
simulationDur = int(np.ceil(simulationDur_ms/fRate))
pre_frames    = 2000.0# in ms
pre_frames    = int(np.ceil(pre_frames/fRate))
preStimFrames = round(simulationDur+pre_frames)
analysisWindowDur = 1500 # in ms
analysisWindowDurFrames = int(np.ceil(analysisWindowDur/fRate))
postStimFrame = preStimFrames + analysisWindowDurFrames

for cohort in interestedCohorts:
    for trainedLevel in trainedLevels:
        for responsiveness in responsivenessTypes:
            plotDatas,animalList = pfun.createTrialvsTraceMatrix(stimTypes, 'Visual', cohort, trainedLevel=trainedLevel, condition=responsiveness)
            
            # Initialization for combined data and IDs
            neural_data_combined, stimID_combined,animalID_combined = [], [],[]
            
            for i, data in enumerate(plotDatas):
                segment_data = data[:, preStimFrames:postStimFrame]
                mask = ~np.isnan(segment_data).any(axis=1)
                filtered_data = segment_data[mask]
                filtered_animal = animalList[i][mask]
                neural_data_combined.append(filtered_data)
                animalID_combined.append(filtered_animal)
                
                stim_ids = np.full((len(filtered_data), 3), [i, stimTypes.index(stimTypes[i]), 0])
                stim_ids[:, 2] = stim_ids[:, 0] + stim_ids[:, 1]
                stimID_combined.append(stim_ids)
            
            neural_data = np.vstack(neural_data_combined)
            stimID = np.vstack(stimID_combined)
            animalID = np.hstack(animalID_combined)
            animalID  = animalID.transpose()

            correlation_matrices = []
            linkage_matrices = []
            for i in range(len(stimTypes)):
                data_i = neural_data[stimID[:, 0] == i]
                corr_matrix_i = np.corrcoef(data_i)
                distance_matrix_i = 1 - corr_matrix_i
                Z_i = linkage(distance_matrix_i, method='single')
                correlation_matrices.append(corr_matrix_i)
                linkage_matrices.append(Z_i)
            
            # Save outputs for population analysisd
            savename = f'C:\\Users\\Huriye\\Documents\\code\\clapfcstimulation\\analysis\\crossCorrelation_{cohort}_{trainedLevel}_{responsiveness}.pkl'
            with open(savename, 'wb') as f:
                pickle.dump([linkage_matrices, correlation_matrices,
                             animalID], f)